In [1]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import os
import pandas as pd
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [2]:
input_data_file = "sort_data_v2.txt"
#df = pd.read_csv(os.path.join("..", "Data", input_data_file),sep='\t')
df = pd.read_csv(os.path.join("..", "Data", input_data_file),sep='\t', nrows=150000)

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
#df = df.dropna()
df

,npi,nppes_provider_gender,nppes_entity_code,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,...,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,year,leie_code,leie
0,1720342884,NaN,O,NEW ORLEANS,701122714.0,LA,US,Clinical Laboratory,Y,O,...,N,332603.0,167054.0,167798.0,105.972315,118.707137,103.835534,2014,1128b7,On_List
1,1720342884,NaN,O,NEW ORLEANS,701122714.0,LA,US,Clinical Laboratory,Y,O,...,N,37517.0,18911.0,25628.0,24.629483,95.659038,24.111942,2014,1128b7,On_List
2,1285651638,NaN,O,WEBSTER,775984924.0,TX,US,Clinical Laboratory,Y,O,...,N,97872.0,14035.0,96892.0,4.999969,7.000000,4.897617,2016,1128b7,On_List
3,1720342884,NaN,O,NEW ORLEANS,701122714.0,LA,US,Clinical Laboratory,Y,O,...,N,108896.0,19431.0,27266.0,26.539299,108.557999,25.983498,2014,1128b7,On_List
4,1285651638,NaN,O,WEBSTER,775984924.0,TX,US,Clinical Laboratory,Y,O,...,N,586053.0,14367.0,99032.0,0.999702,2.824145,0.979123,2016,1128b7,On_List
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,1396830204,NaN,O,SANTA FE SPRINGS,906706000.0,CA,US,Clinical Laboratory,Y,O,...,N,4373.0,3506.0,4373.0,17.620000,31.405760,17.262902,2015,None,Not
149996,1437297298,M,I,JOSHUA TREE,92252.0,CA,US,Cardiology,Y,F,...,N,3288.0,2421.0,2828.0,8.693464,34.800365,6.818634,2015,None,Not
149997,1629232491,M,I,SAINT LOUIS,631366163.0,MO,US,Pathology,Y,O,...,N,1993.0,1126.0,1383.0,44.437968,306.000000,32.658981,2015,None,Not
149998,1699782722,NaN,O,NASHVILLE,372281310.0,TN,US,Clinical Laboratory,Y,O,...,N,3369.0,2502.0,3368.0,23.420000,85.000000,22.875741,2015,None,Not


In [3]:
df['nppes_provider_gender'] = df['nppes_provider_gender'].fillna('O')

In [4]:
df.columns

Index(['npi', 'nppes_provider_gender', 'nppes_entity_code',
       'nppes_provider_city', 'nppes_provider_zip', 'nppes_provider_state',
       'nppes_provider_country', 'provider_type',
       'medicare_participation_indicator', 'place_of_service', 'hcpcs_code',
       'hcpcs_description', 'hcpcs_drug_indicator', 'line_srvc_cnt',
       'bene_unique_cnt', 'bene_day_srvc_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_payment_amt', 'year',
       'leie_code', 'leie'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 22 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   npi                               150000 non-null  int64  
 1   nppes_provider_gender             150000 non-null  object 
 2   nppes_entity_code                 150000 non-null  object 
 3   nppes_provider_city               150000 non-null  object 
 4   nppes_provider_zip                150000 non-null  float64
 5   nppes_provider_state              150000 non-null  object 
 6   nppes_provider_country            150000 non-null  object 
 7   provider_type                     150000 non-null  object 
 8   medicare_participation_indicator  150000 non-null  object 
 9   place_of_service                  150000 non-null  object 
 10  hcpcs_code                        150000 non-null  object 
 11  hcpcs_description                 150000 non-null  o

# Select your features (columns)

In [6]:
# Set features. This will also be used as your x values.
selected_features = df[['line_srvc_cnt', 'place_of_service', 'hcpcs_drug_indicator', 'nppes_entity_code', 'nppes_provider_gender',
                       'medicare_participation_indicator', 'leie']]
selected_features = selected_features.dropna()
selected_features

,line_srvc_cnt,place_of_service,hcpcs_drug_indicator,nppes_entity_code,nppes_provider_gender,medicare_participation_indicator,leie
0,332603.0,O,N,O,O,Y,On_List
1,37517.0,O,N,O,O,Y,On_List
2,97872.0,O,N,O,O,Y,On_List
3,108896.0,O,N,O,O,Y,On_List
4,586053.0,O,N,O,O,Y,On_List
...,...,...,...,...,...,...,...
149995,4373.0,O,N,O,O,Y,Not
149996,3288.0,F,N,I,M,Y,Not
149997,1993.0,O,N,I,M,Y,Not
149998,3369.0,O,N,O,O,Y,Not


# Create a Train Test Split

Use `koi_disposition` for the y values

In [7]:
selected_features.info()

#output_data_file = "mike_look_test.txt"
#selected_features.to_csv(os.path.join("..", "Data", output_data_file), index= False, sep='\t')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 0 to 149999
Data columns (total 7 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   line_srvc_cnt                     150000 non-null  float64
 1   place_of_service                  150000 non-null  object 
 2   hcpcs_drug_indicator              150000 non-null  object 
 3   nppes_entity_code                 150000 non-null  object 
 4   nppes_provider_gender             150000 non-null  object 
 5   medicare_participation_indicator  150000 non-null  object 
 6   leie                              150000 non-null  object 
dtypes: float64(1), object(6)
memory usage: 9.2+ MB


In [8]:
disposition = selected_features["leie"]
disposition_names = ["Not", "On_List"]
selected_features = selected_features.drop("leie", axis=1)
feature_names = selected_features.columns
selected_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 0 to 149999
Data columns (total 6 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   line_srvc_cnt                     150000 non-null  float64
 1   place_of_service                  150000 non-null  object 
 2   hcpcs_drug_indicator              150000 non-null  object 
 3   nppes_entity_code                 150000 non-null  object 
 4   nppes_provider_gender             150000 non-null  object 
 5   medicare_participation_indicator  150000 non-null  object 
dtypes: float64(1), object(5)
memory usage: 8.0+ MB


In [9]:
mike_df2 = disposition
mike3 = mike_df2.to_frame()
mike3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 0 to 149999
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   leie    150000 non-null  object
dtypes: object(1)
memory usage: 2.3+ MB


In [10]:
mike3.leie.value_counts()

Not        89678
On_List    60322
Name: leie, dtype: int64

In [11]:
data = pd.get_dummies(selected_features)
data

,line_srvc_cnt,place_of_service_F,place_of_service_O,hcpcs_drug_indicator_N,hcpcs_drug_indicator_Y,nppes_entity_code_I,nppes_entity_code_O,nppes_provider_gender_F,nppes_provider_gender_M,nppes_provider_gender_O,medicare_participation_indicator_Y
0,332603.0,0,1,1,0,0,1,0,0,1,1
1,37517.0,0,1,1,0,0,1,0,0,1,1
2,97872.0,0,1,1,0,0,1,0,0,1,1
3,108896.0,0,1,1,0,0,1,0,0,1,1
4,586053.0,0,1,1,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
149995,4373.0,0,1,1,0,0,1,0,0,1,1
149996,3288.0,1,0,1,0,1,0,0,1,0,1
149997,1993.0,0,1,1,0,1,0,0,1,0,1
149998,3369.0,0,1,1,0,0,1,0,0,1,1


In [12]:
data = data.drop(['nppes_entity_code_O', 'medicare_participation_indicator_Y', 'place_of_service_O', 'hcpcs_drug_indicator_Y'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 0 to 149999
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   line_srvc_cnt            150000 non-null  float64
 1   place_of_service_F       150000 non-null  uint8  
 2   hcpcs_drug_indicator_N   150000 non-null  uint8  
 3   nppes_entity_code_I      150000 non-null  uint8  
 4   nppes_provider_gender_F  150000 non-null  uint8  
 5   nppes_provider_gender_M  150000 non-null  uint8  
 6   nppes_provider_gender_O  150000 non-null  uint8  
dtypes: float64(1), uint8(6)
memory usage: 3.1 MB


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, disposition, random_state=42)

In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112500 entries, 92988 to 121958
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   line_srvc_cnt            112500 non-null  float64
 1   place_of_service_F       112500 non-null  uint8  
 2   hcpcs_drug_indicator_N   112500 non-null  uint8  
 3   nppes_entity_code_I      112500 non-null  uint8  
 4   nppes_provider_gender_F  112500 non-null  uint8  
 5   nppes_provider_gender_M  112500 non-null  uint8  
 6   nppes_provider_gender_O  112500 non-null  uint8  
dtypes: float64(1), uint8(6)
memory usage: 2.4 MB


In [15]:
y_train

92988         Not
43658     On_List
110698        Not
77130         Not
109216        Not
           ...   
119879        Not
103694        Not
131932        Not
146867        Not
121958        Not
Name: leie, Length: 112500, dtype: object

In [16]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')

# Pre-processing
Scale the data using the MinMaxScaler and perform some feature selection

In [17]:
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)

X_train_scale = X_minmax.transform(X_train)
X_test_scale = X_minmax.transform(X_test)


In [18]:
X_train_scale

array([[1.07586680e-03, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [2.38710184e-07, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [1.17922831e-03, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       ...,
       [1.11477656e-03, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [8.57208271e-04, 0.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.27389065e-04, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

# Hyperparameter Tuning
Use GridSearchCV to tune the model's parameters

In [19]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 2],
              'gamma': [0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [20]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train_scale, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] C=1, gamma=0.001 ................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................... C=1, gamma=0.001, score=0.820, total= 1.7min
[CV] C=1, gamma=0.001 ................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min remaining:    0.0s


[CV] .................... C=1, gamma=0.001, score=0.813, total= 2.2min
[CV] C=1, gamma=0.001 ................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.9min remaining:    0.0s


[CV] .................... C=1, gamma=0.001, score=0.821, total= 1.9min
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.819, total= 2.9min
[CV] C=1, gamma=0.001 ................................................
[CV] .................... C=1, gamma=0.001, score=0.813, total= 1.8min
[CV] C=1, gamma=0.01 .................................................
[CV] ..................... C=1, gamma=0.01, score=0.820, total= 1.9min
[CV] C=1, gamma=0.01 .................................................
[CV] ..................... C=1, gamma=0.01, score=0.813, total= 2.2min
[CV] C=1, gamma=0.01 .................................................
[CV] ..................... C=1, gamma=0.01, score=0.821, total= 2.6min
[CV] C=1, gamma=0.01 .................................................
[CV] ..................... C=1, gamma=0.01, score=0.819, total= 2.4min
[CV] C=1, gamma=0.01 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 37.8min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 2], 'gamma': [0.001, 0.01]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [21]:
print(f"Training Data Score: {grid.score(X_train_scale, y_train)}")
print(f"Testing Data Score: {grid.score(X_test_scale, y_test)}")

Training Data Score: 0.8171288888888889
Testing Data Score: 0.8158933333333334


In [22]:
# List the best parameters for this dataset
print(grid.best_params_)

{'C': 2, 'gamma': 0.001}


In [23]:
# List the best score
print(grid.best_score_)

0.8171022222222222


In [24]:
type(grid.cv_results_)

dict

In [25]:
print(grid.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_C', 'param_gamma', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [26]:
import pandas as pd

cv_results = pd.DataFrame(grid.cv_results_)
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,118.810928,26.957751,7.027053,1.876008,1,0.001,"{'C': 1, 'gamma': 0.001}",0.82,0.812933,0.821022,0.818711,0.812800,0.817093,0.003528,3
1,122.646501,17.210783,7.224562,1.807430,1,0.01,"{'C': 1, 'gamma': 0.01}",0.82,0.812933,0.821022,0.818711,0.812800,0.817093,0.003528,3
2,93.990203,10.878898,5.921969,0.201021,2,0.001,"{'C': 2, 'gamma': 0.001}",0.82,0.812889,0.821022,0.818711,0.812889,0.817102,0.003517,1
3,91.667618,11.304337,5.833402,0.183513,2,0.01,"{'C': 2, 'gamma': 0.01}",0.82,0.812889,0.821022,0.818711,0.812889,0.817102,0.003517,1


In [27]:
cv_results_tiny = cv_results[['param_C', 'param_gamma', 'mean_test_score']]
cv_results_tiny.sort_values(by='mean_test_score', ascending=False).head(50)

,param_C,param_gamma,mean_test_score
2,2,0.001,0.817102
3,2,0.01,0.817102
0,1,0.001,0.817093
1,1,0.01,0.817093


In [28]:
# Best index for the model to impact the model.
print(grid.best_index_)

2


In [29]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scale)

In [30]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["Not", "On_list"]))

              precision    recall  f1-score   support

         Not       0.96      0.72      0.82     22467
     On_list       0.70      0.96      0.81     15033

    accuracy                           0.82     37500
   macro avg       0.83      0.84      0.82     37500
weighted avg       0.86      0.82      0.82     37500



In [31]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'Medicare_SVC_Grid_10-21-20_V2.sav'
joblib.dump(grid, filename)

['Medicare_SVC_Grid_10-21-20_V2.sav']

In [32]:
#from sklearn.metrics import f1_score

#f1_score(y_test, y_train, average='weighted', labels=np.unique(y_train))
#f1_score(y_test, y_train, average='weighted')

In [33]:
#from sklearn.metrics import f1_score

In [34]:
#predictions